In [9]:
!nvidia-smi

Mon Nov 28 20:04:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [12]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [17]:
! kaggle datasets download -d shayanriyaz/riceleafs

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [18]:
!unzip "/content/riceleafs.zip"

unzip:  cannot find or open /content/riceleafs.zip, /content/riceleafs.zip.zip or /content/riceleafs.zip.ZIP.


# Data Augmentation

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SHAPE=(224,224)
BATCH_SIZE=32

train_dir="/content/RiceLeafs/train"
test_dir="/content/RiceLeafs/validation"

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode="nearest")

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMG_SHAPE,
    batch_size=64)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    shuffle=False, 
    seed=42,
    color_mode="rgb",     
    class_mode="categorical",
    target_size=IMG_SHAPE,
    batch_size=BATCH_SIZE)

# Building Model
### Functional API Model Building

In [ ]:
import tensorflow as tf
#input layer
inputs=tf.keras.layers.Input(shape=(224,224,3))
x=feature_vectorizer(inputs) #2d 
x=tf.keras.layers.Flatten()(x) #1d

#Hidden layer
x=tf.keras.layers.Dense(512, activation='relu')(x)
x=tf.keras.layers.Dense(256, activation='relu')(x)
x=tf.keras.layers.Dense(128, activation='relu')(x)


x=tf.keras.layers.Dropout(0.2)(x) #avoids overfitting

#Output layer
outputs=tf.keras.layers.Dense(train_generator.num_classes,activation="softmax")(x)

#Model setting
model1=tf.keras.Model(inputs,outputs)
model1.summary()

# Compile the model
model1.compile(loss="categorical_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])

# Fitting the model
model1.fit(train_generator,
           epochs=5 ,                 #Epochs
           steps_per_epoch=train_generator.samples//train_generator.batch_size,
           validation_data=test_generator,
           validation_steps=test_generator.samples//test_generator.batch_size)


# New Section

# New Section

In [ ]:
#imagenet feature vectorizer
import tensorflow_hub as hub
feature_vectorizer = hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5",
                                   input_shape=IMG_SHAPE+(3,))

# Image Loading and Predictions

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import tensorflow as tf
import io
def load_and_prep(filename,img_shape=224,scale=False):
  # reading the image
  img=tf.io.read_file(filename)

  # Decode the image into tensors
  img=tf.image.decode_image(img)

  # Resize the image 
  img=tf.image.resize(img,[img_shape,img_shape])

  # Scale yes/no?
  img=img/255.
 
  return img

In [ ]:
class_names=["BrownSpot","Healthy","Hispa","LeafBlast"]

import matplotlib.pyplot as plt
import numpy as np

def pred_and_plot(model,filename,class_names=class_names):
  # Import the target image and preprocess it.
  img=load_and_prep(filename)

  # Make predictions 
  pred=model.predict(tf.expand_dims(img,axis=0))

  # Get the predicted class
  pred_class=class_names[int(np.argmax(pred))]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"prediction: {pred_class}")
  plt.axis(False);

In [ ]:
pred_and_plot(model1,"/content/rli1.jpg")